In [2]:
from loguru import logger
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import json
import pandas as pd
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9999")
driver = webdriver.Chrome(options=options)

In [4]:
url = "https://www.bing.com/images/create/"
driver.get(url)

In [15]:

# visited_urls = []
class_elements = driver.find_elements(By.XPATH, '//input[contains(@class, "b_searchbox gi_sb")]')
class_elements

[<selenium.webdriver.remote.webelement.WebElement (session="4ab375b8c29df48a558a26666c50c7d2", element="3232390F996F49BA3C6DF4B9D00EC4BB_element_136")>]

In [17]:
# class_elements[0].click()
class_elements[0].send_keys('Testing\n')

In [49]:
# Find the div containing the images
div_element = driver.find_elements(By.ID, "mmComponent_images_as_1")

# Locate all img tags within the div
images = div_element[0].find_elements(By.TAG_NAME, "img")
print(images)

IndexError: list index out of range

In [60]:
driver.switch_to.default_content()
for idx, image in enumerate(images):
    image.click()
    time.sleep(5)
    driver.switch_to.frame("OverlayIFrame")

    # Locate the large image
    large_image_element = driver.find_elements(By.XPATH, '//div[contains(@class, "imgContainer")]')
    for image_element in large_image_element:
        if '<img' in image_element.get_attribute('innerHTML'):
            break

    # Get the src attribute of the large image
    src = image_element.get_attribute('innerHTML').split('src="')[-1].split('"')[0]

    # Download and save the image
    response = requests.get(src, stream=True)
    with open(f'image_{idx}.png', 'wb') as out_file:
        out_file.write(response.content)

    driver.switch_to.default_content()

    # Navigate back to the thumbnails
    driver.back()

In [22]:
for idx, image in enumerate(images):
    src = image.get_attribute("src")
    response = requests.get(src, stream=True)
    with open(f'image_{idx}.png', 'wb') as out_file:
        out_file.write(response.content)
